In [1]:
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/Technical/AI ML Model/Dataset/dataset.csv')
dataset
columns_to_drop = ['Index', 'Air Temperature', 'Water Temperature', 'Month', 'Day', 'Time of Day']
dataset = dataset.drop(columns=columns_to_drop)
dataset
dataset.isnull().sum()
columns_to_impute = ["pH", "Iron", "Nitrate", "Chloride", "Lead", "Zinc", "Turbidity", "Fluoride", "Copper", "Odor", "Sulfate", "Conductivity", "Chlorine","Manganese", "Total Dissolved Solids"]
high_frequency_columns = ["Color", "Source"]
for col in high_frequency_columns:
    most_frequent_value = dataset[col].mode().values[0]
    dataset[col].fillna(most_frequent_value, inplace=True)
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
dataset[columns_to_impute] = imputer.fit_transform(dataset[columns_to_impute])
dataset.isnull().sum()
unique_values = dataset['Color'].unique()
unique_values
color = {'Colorless': 0,'Faint Yellow': 1,'Light Yellow': 2,'Near Colorless': 3,'Yellow':4}
dataset['Color'] = dataset['Color'].map(color)
dataset
unique_values = dataset['Source'].unique()
unique_values
source = {'Stream': 0,'Lake': 1,'River': 2,'Ground': 3,'Spring': 4,'Aquifer': 5,'Reservoir': 6,'Well': 7}
dataset['Source'] = dataset['Source'].map(source)
dataset
X = dataset.iloc[:, 0:-2]
y = dataset.iloc[:, -1]
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [3]:
X_train_scaled = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)
X_test_scaled = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

In [4]:
class NeuralNet(nn.Module):
    def __init__(self, input_shape):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_shape, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
model = NeuralNet(X_train_scaled.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [6]:
import time
start_timept = time.time()

In [7]:
train_data = TensorDataset(X_train_scaled, y_train)
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
val_data = TensorDataset(X_val, y_val)
val_loader = DataLoader(dataset=val_data, batch_size=32)

In [8]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        outputs = model(batch_x)
        loss = criterion(outputs.squeeze(), batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for batch_x, batch_y in val_loader:
            outputs = model(batch_x)
            val_loss += criterion(outputs.squeeze(), batch_y).item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Loss: {val_loss/len(val_loader):.4f}')

Epoch [1/10], Loss: 0.0853, Validation Loss: 0.0903
Epoch [2/10], Loss: 0.0529, Validation Loss: 0.0897
Epoch [3/10], Loss: 0.0877, Validation Loss: 0.0891
Epoch [4/10], Loss: 0.0837, Validation Loss: 0.0888
Epoch [5/10], Loss: 0.0900, Validation Loss: 0.0889
Epoch [6/10], Loss: 0.0623, Validation Loss: 0.0885
Epoch [7/10], Loss: 0.1190, Validation Loss: 0.0884
Epoch [8/10], Loss: 0.0415, Validation Loss: 0.0885
Epoch [9/10], Loss: 0.0546, Validation Loss: 0.0885
Epoch [10/10], Loss: 0.0755, Validation Loss: 0.0883


In [9]:
end_timept = time.time()
runtimept = end_timept - start_timept
print(f"Total Runtime: {runtimept:.2f} seconds")

Total Runtime: 2581.24 seconds


In [10]:
model.eval()
with torch.no_grad():
    outputs = model(X_test_scaled)
    mse = criterion(outputs.squeeze(), y_test)
print(f"Mean Squared Error on Test Set: {mse.item()}")

Mean Squared Error on Test Set: 0.08758249878883362


In [11]:
threshold = 0.5
correct = 0
total = y_test.size(0)
model.eval()
with torch.no_grad():
    outputs = model(X_test_scaled)
    diff = torch.abs(outputs.squeeze() - y_test)
    correct = (diff < threshold).sum().item()
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')

Accuracy: 88.13%
